# Merging Survey and Platform Data -  Group with Randomization

Completely done in Python. There ore no steps required in Excel

v2_18.04.2024

In [1]:
import pandas as pd

In [2]:
# Load the first dataset
df1 = pd.read_csv('data_prep_validation_pre-merge_survey.csv',  sep=';')

# Load the second dataset
df2 = pd.read_csv('data_prep_cleaned_platform.csv',  sep=';')

In [3]:
# Merge the datasets on 'VPNCode'
merged_df = pd.merge(df1, df2[['VPNCode', 'group_nr']], on='VPNCode', how='inner')

In [4]:
# Treat 'group_nr' as a categorical variable
merged_df['group_nr'] = merged_df['group_nr'].astype('category')

In [5]:
merged_df.head()

,VPNCode,AILiteracy[Use1],AILiteracy[Use2],AILiteracy[Use3],AILiteracy[Use4],AILiteracy[Use5],AILiteracy[Use6],AILiteracy[Kno1],AILiteracy[Kno2],AILiteracy[Kno3],...,NEO[E],NEO[A],NEO[C],NEO[N],NEO[O],JC1[IStR],JC1[HRJD],JC2[IStR],JC2[HRJD],group_nr
0,ZZEBA0308,5,5,5,5,5,5,4,5,4,...,4.25,2.75,5.00,4.25,4.0,3.6,5.000000,2.2,1.75,6
1,ZZNRO406,2,4,4,4,4,2,5,3,5,...,3.00,3.25,3.75,3.25,4.0,2.4,3.333333,3.8,4.00,4
2,ZZSSA0301,5,5,5,5,5,5,5,5,5,...,3.50,3.00,4.00,2.25,4.0,3.2,2.333333,4.2,4.25,5
3,ZZAWA0212,5,5,5,5,5,5,5,5,5,...,3.25,2.50,4.00,2.75,4.4,2.4,5.000000,2.2,5.00,3
4,ZZKO1608,5,4,4,4,4,4,3,3,3,...,2.75,2.50,3.25,2.50,2.8,3.8,3.333333,3.6,3.50,2


In [7]:
# Adress Radomization by exchaning values from the following columns if the 'group_nr' values are 5, 6, 7, and 8 (from 1 to 4 the order is already correct)
# Define the columns to swap in pairs
swap_pairs = [
    ('JC1[IStR1]', 'JC2[2IStR1]'),
    ('JC1[IStR2]', 'JC2[2IStR2]'),
    ('JC1[IStR3]', 'JC2[2IStR3]'),
    ('JC1[IStR4]', 'JC2[2IStR4]'),
    ('JC1[IStR5]', 'JC2[2IStR5]'),
    ('JC1[HRJD1]', 'JC2[2HRJD1]'),
    ('JC1[HRJD2]', 'JC2[2HRJD2]'),
    ('JC1[HRJD3]', 'JC2[2HRJD5]'),
    ('JC1[HRJD]', 'JC2[HRJD]'),
    ('JC1[IStR]', 'JC2[IStR]'),
]

# Loop through each pair and swap their values for rows with group_nr in 5, 6, 7, 8
for col1, col2 in swap_pairs:
    # Identify rows where swapping is needed
    mask = merged_df['group_nr'].isin([5, 6, 7, 8])
    
    # Perform the swap
    merged_df.loc[mask, [col1, col2]] = merged_df.loc[mask, [col2, col1]].values

In [8]:
# Drop group_nr as it is no longer needed
merged_df.drop(columns=['group_nr'], inplace=True)

In [9]:
# Renaming columns to remove brackets
merged_df.columns = [col.replace('[', '').replace(']', '') for col in merged_df.columns]

In [10]:
# Look at the new dataset
merged_df.head()

,VPNCode,AILiteracyUse1,AILiteracyUse2,AILiteracyUse3,AILiteracyUse4,AILiteracyUse5,AILiteracyUse6,AILiteracyKno1,AILiteracyKno2,AILiteracyKno3,...,CMV,NEOE,NEOA,NEOC,NEON,NEOO,JC1IStR,JC1HRJD,JC2IStR,JC2HRJD
0,ZZEBA0308,5,5,5,5,5,5,4,5,4,...,3.666667,4.25,2.75,5.00,4.25,4.0,2.2,1.750000,3.6,5.000000
1,ZZNRO406,2,4,4,4,4,2,5,3,5,...,3.666667,3.00,3.25,3.75,3.25,4.0,2.4,3.333333,3.8,4.000000
2,ZZSSA0301,5,5,5,5,5,5,5,5,5,...,5.000000,3.50,3.00,4.00,2.25,4.0,4.2,4.250000,3.2,2.333333
3,ZZAWA0212,5,5,5,5,5,5,5,5,5,...,2.666667,3.25,2.50,4.00,2.75,4.4,2.4,5.000000,2.2,5.000000
4,ZZKO1608,5,4,4,4,4,4,3,3,3,...,3.666667,2.75,2.50,3.25,2.50,2.8,3.8,3.333333,3.6,3.500000


In [11]:
# Save DF in a new CSV-File 
merged_df.to_csv('data_prep_validation_prolific.csv', index=False, encoding='utf-8-sig', sep=',')